In [1]:
cpdb_file_path = "/Users/uqztuong/Documents/GitHub/CellphoneDB/notebooks/cpdb_output/db_inputs/cellphonedb_07_31_2023_211630.zip"
meta_file_path = "/Users/uqztuong/Documents/GitHub/ktplotspy/data/meta.txt"
counts_file_path = "/Users/uqztuong/Documents/GitHub/ktplotspy/data/kidneyimmune.h5ad"
degs_file_path = "/Users/uqztuong/Documents/GitHub/ktplotspy/data/degs.tsv"
out_path_degs = "/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs"
out_path_stat = "/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat"

In [2]:
import scanpy as sc

adata = sc.read_h5ad(counts_file_path)
adata

AnnData object with n_obs × n_vars = 1000 × 19969
    obs: 'celltype', 'compartment', 'Experiment', 'Project', 'n_genes', 'percent_mito', 'n_counts', 'Experiment_celltype', 'ident'
    obsm: 'X_pca', 'X_umap'
    layers: 'logcounts'

In [3]:
import pandas as pd

df = pd.DataFrame(adata.obs["celltype"]).reset_index()
df.columns = ["barcode_sample", "cell_type"]
df.to_csv(meta_file_path, sep="\t", index=False)

In [4]:
df

,barcode_sample,cell_type
0,4602STDY7090428_CGTGTCTAGTCACGCC-1,CD4T cell
1,CZIKidney7587405_GTGCTTCCACACCGAC-1,MNPa(mono)
2,4602STDY6930852_ACATACGAGATATGCA-1,NKT cell
3,4602STDY6976427_TCTATTGAGACAATAC-1,Neutrophil
4,4602STDY6930853_CATCGAAAGATGTTAG-1,MNPc(DC)
...,...,...
995,4602STDY6949181_CGAGAAGGTAAGGGAA-1,MNPa(mono)
996,4602STDY6976427_TCAACGATCGGCTACG-1,NKT cell
997,CZIKidney7632803_TTCTCCTTCCCAAGAT-1,B cell
998,CZIKidney7587406_CGTTGGGCAATGAATG-1,CD4T cell


In [5]:
sc.tl.rank_genes_groups(adata, groupby="celltype", method="wilcoxon")

In [6]:
# Conver to dense matrix for Seurat
adata.X = adata.X.toarray()

In [7]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython


In [8]:

%%R -i adata
adata

/opt/homebrew/Caskroom/mambaforge/base/envs/ktplots/lib/python3.11/functools.py:909: NotConvertedWarning: Conversion 'py2rpy' not defined for objects of type '<class 'NoneType'>'
  return dispatch(args[0].__class__)(*args, **kw)


class: SingleCellExperiment 
dim: 19969 1000 
metadata(1): rank_genes_groups
assays(2): X logcounts
rownames(19969): RP11-34P13.7 FO538757.2 ... AC004556.1 AC240274.1
rowData names(0):
colnames(1000): 4602STDY7090428_CGTGTCTAGTCACGCC-1
  CZIKidney7587405_GTGCTTCCACACCGAC-1 ...
  CZIKidney7587406_CGTTGGGCAATGAATG-1
  4602STDY7018630_GTGTGCGTCGGAAACG-1
colData names(9): celltype compartment ... Experiment_celltype ident
reducedDimNames(2): PCA UMAP
mainExpName: NULL
altExpNames(0):


In [9]:
%%R
library(Seurat)
so = as.Seurat(adata, counts = "X", data = "X")
Idents(so) = so$celltype


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)
Attaching SeuratObject

Attaching package: ‘Seurat’

The following object is masked from ‘package:SummarizedExperiment’:

    Assays



In [10]:
%%R -o DEGs

## OPTION 1 - compute DEGs for all cell types
## Extract DEGs for each cell_type
DEGs <- FindAllMarkers(so, 
                       test.use = 'LR', 
                       verbose = F, 
                       only.pos = T, 
                       random.seed = 1, 
                       logfc.threshold = 0.2, 
                       min.pct = 0.1, 
                       return.thresh = 0.05)

In addition: Warning messages:
1: glm.fit: fitted probabilities numerically 0 or 1 occurred 
2: glm.fit: fitted probabilities numerically 0 or 1 occurred 
3: glm.fit: fitted probabilities numerically 0 or 1 occurred 
4: glm.fit: fitted probabilities numerically 0 or 1 occurred 
5: glm.fit: fitted probabilities numerically 0 or 1 occurred 
6: glm.fit: fitted probabilities numerically 0 or 1 occurred 
7: glm.fit: fitted probabilities numerically 0 or 1 occurred 
8: glm.fit: fitted probabilities numerically 0 or 1 occurred 


In [11]:
DEGs.head()

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
LYZ,5.629062e-144,1.542476,0.975,0.276,1.124067e-139,MNPa(mono),LYZ
S100A9,2.132182e-126,1.785355,0.905,0.149,4.257754e-122,MNPa(mono),S100A9
VCAN,1.476268e-118,1.454228,0.760,0.044,2.947959e-114,MNPa(mono),VCAN
FCN1,1.039374e-112,1.495729,0.893,0.124,2.075527e-108,MNPa(mono),FCN1
S100A8,1.765145e-111,1.804471,0.860,0.100,3.524819e-107,MNPa(mono),S100A8


In [12]:
cond1 = DEGs["p_val_adj"] < 0.05
cond2 = DEGs["avg_log2FC"] > 0.1
mask = [all(tup) for tup in zip(cond1, cond2)]
fDEGs = DEGs[mask]

In [13]:
# 1st column = cluster; 2nd column = gene
fDEGs = fDEGs[["cluster", "gene", "p_val_adj", "p_val", "avg_log2FC", "pct.1", "pct.2"]]
fDEGs.to_csv(degs_file_path, index=False, sep="\t")

In [15]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method

results3 = cpdb_degs_analysis_method.call(
    cpdb_file_path=cpdb_file_path,  # mandatory: CellPhoneDB database zip file.
    meta_file_path=meta_file_path,  # mandatory: tsv file defining barcodes to cell label.
    counts_file_path=counts_file_path,  # mandatory: normalized count matrix.
    degs_file_path=degs_file_path,  # mandatory: tsv file with DEG to account.
    counts_data="hgnc_symbol",  # defines the gene annotation in counts matrix.
    output_path=out_path_degs,  # Path to save results.
)

[ ][CORE][01/11/23-09:27:32][INFO] [Cluster DEGs Analysis] Threshold:0.1 Precision:3
Reading user files...
The following user files were loaded successfully:
/Users/uqztuong/Documents/GitHub/ktplotspy/data/kidneyimmune.h5ad
/Users/uqztuong/Documents/GitHub/ktplotspy/data/meta.txt
/Users/uqztuong/Documents/GitHub/ktplotspy/data/degs.tsv
[ ][CORE][01/11/23-09:27:34][INFO] Running Real Analysis
[ ][CORE][01/11/23-09:27:34][INFO] Running DEGs-based Analysis
[ ][CORE][01/11/23-09:27:34][INFO] Building results
Saved deconvoluted_result to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs/degs_analysis_deconvoluted_result_11_01_2023_092734.txt
Saved means_result to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs/degs_analysis_means_result_11_01_2023_092734.txt
Saved relevant_interactions_result to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs/degs_analysis_relevant_interactions_result_11_01_2023_092734.txt
Saved significant_means to /Users/uqztuong/Document

In [16]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

results3 = cpdb_statistical_analysis_method.call(
    cpdb_file_path=cpdb_file_path,  # mandatory: CellPhoneDB database zip file.
    meta_file_path=meta_file_path,  # mandatory: tsv file defining barcodes to cell label.
    counts_file_path=counts_file_path,  # mandatory: normalized count matrix.
    counts_data="hgnc_symbol",  # defines the gene annotation in counts matrix.
    output_path=out_path_stat,  # Path to save results.
)

Reading user files...
The following user files were loaded successfully:
/Users/uqztuong/Documents/GitHub/ktplotspy/data/kidneyimmune.h5ad
/Users/uqztuong/Documents/GitHub/ktplotspy/data/meta.txt
[ ][CORE][01/11/23-09:27:42][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:-1 Threads:4 Precision:3
[ ][CORE][01/11/23-09:27:42][INFO] Running Real Analysis
[ ][CORE][01/11/23-09:27:42][INFO] Running Statistical Analysis


100%|██████████| 1000/1000 [00:08<00:00, 118.56it/s]

[ ][CORE][01/11/23-09:27:50][INFO] Building Pvalues result


[ ][CORE][01/11/23-09:27:50][INFO] Building results
Saved deconvoluted to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_deconvoluted_11_01_2023_092750.txt
Saved means to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_means_11_01_2023_092750.txt
Saved pvalues to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_pvalues_11_01_2023_092750.txt
Saved significant_means to /Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_significant_means_11_01_2023_092750.txt


In [19]:
%%R
decon_degs <- read.delim("/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs/degs_analysis_deconvoluted_result_11_01_2023_092734.txt", check.names=FALSE)
means_degs <- read.delim("/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs/degs_analysis_means_result_11_01_2023_092734.txt", check.names=FALSE)
rel_int_degs <- read.delim("/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_degs/degs_analysis_relevant_interactions_result_11_01_2023_092734.txt", check.names=FALSE)
save(means_degs, rel_int_degs, decon_degs, file = "/Users/uqztuong/Documents/GitHub/ktplots/data/cpdb_output_degs.RData")

In [21]:
%%R
decon_stat <- read.delim("/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_deconvoluted_11_01_2023_092750.txt", check.names=FALSE)
means_stat <- read.delim("/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_means_11_01_2023_092750.txt", check.names=FALSE)
pvals_stat <- read.delim("/Users/uqztuong/Documents/GitHub/ktplotspy/data/out_v5_stat/statistical_analysis_pvalues_11_01_2023_092750.txt", check.names=FALSE)
save(means_stat, pvals_stat, decon_stat, file = "/Users/uqztuong/Documents/GitHub/ktplots/data/cpdb_output_stat.RData")